In [1]:
import calendar
import json
import os
import time
import traceback
from datetime import datetime

import pandas as pd
import requests
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

spark = (
    SparkSession.builder
        .master("local")
        .appName("TFL Notebook")
        .getOrCreate()
)

trips = spark.read.parquet("data/parquet")

trips.limit(10).toPandas()


,rental_id,duration,bike_id,end_date,end_station_id,end_station_name,start_date,start_station_id,start_station_name
0,41866884,600,8876,16/03/2015 00:00,410,"Edgware Road Station, Paddington",15/03/2015 23:50,6,"Broadcasting House, Marylebone"
1,41866889,480,9384,16/03/2015 00:00,580,"Doddington Grove, Kennington",15/03/2015 23:52,94,"Bricklayers Arms, Borough"
2,41866836,1920,12180,16/03/2015 00:01,664,"Austin Road, Battersea",15/03/2015 23:29,194,"Hop Exchange, The Borough"
3,41866853,1440,8468,16/03/2015 00:01,693,"Felsham Road, Putney",15/03/2015 23:37,687,"Maclise Road, Olympia"
4,41866894,420,11789,16/03/2015 00:02,28,"Bolsover Street, Fitzrovia",15/03/2015 23:55,162,"Southampton Place, Holborn"
5,41866909,180,2797,16/03/2015 00:03,294,"St. George's Square, Pimlico",16/03/2015 00:00,268,"Belgrave Road, Victoria"
6,41866899,300,9999,16/03/2015 00:03,675,"Usk Road, Battersea",15/03/2015 23:58,701,"Vicarage Crescent, Battersea"
7,41866886,780,6028,16/03/2015 00:04,205,"New Road 2, Whitechapel",15/03/2015 23:51,717,"Dunston Road , Haggerston"
8,41866895,540,6555,16/03/2015 00:04,125,"Borough High Street, The Borough",15/03/2015 23:55,64,"William IV Street, Strand"
9,41866881,900,5674,16/03/2015 00:05,185,"Alderney Street, Pimlico",15/03/2015 23:50,229,"Whitehall Place, Strand"


In [3]:
df_1 = (
    trips
    .groupby('start_station_name')
    .agg({
        "*": "count",
        "duration": "avg",
    })
)

df_1.toPandas()

,start_station_name,avg(duration),count(1)
0,"Dunston Road , Haggerston",1449.487560,50523
1,"George Place Mews, Marylebone",1089.242731,19293
2,"Panton Street, West End",1697.574452,32034
3,"Limerston Street, West Chelsea",1199.971528,40039
4,"Park Lane, Mayfair",1868.823090,20061
5,"Russell Gardens, Holland Park",988.986906,1451
6,"Portland Place, Marylebone",1088.498327,30486
7,"Montgomery Square, Canary Wharf",1492.336595,15330
8,"Appold Street, Liverpool Street",985.731392,27058
9,"Kingsway Southbound, Strand",1191.666845,28023
